In [13]:
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px

### 유저 데이터

In [14]:
users_df = pd.read_csv("../datasets/users.csv")
users_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13784085 entries, 0 to 13784084
Data columns (total 3 columns):
 #   Column    Dtype
---  ------    -----
 0   user_id   int64
 1   products  int64
 2   reviews   int64
dtypes: int64(3)
memory usage: 315.5 MB


In [15]:
users_df.head(10)

,user_id,products,reviews
0,7089523,359,0
1,13509147,156,1
2,8443010,329,4
3,4631165,176,3
4,4967668,98,2
5,5449605,145,4
6,5667011,447,2
7,7013607,1083,1
8,7172377,273,1
9,7188777,51,1


#### users_df는 user_id, products, reviews의 열로 구성되어 있으며 유저의 아이디, 사용자가 구매한 게임 수/추가 기능 수, 리뷰 개수에 해당합니다.

### 게임 데이터

In [16]:
games_df = pd.read_csv("../datasets/games.csv")
games_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50872 entries, 0 to 50871
Data columns (total 13 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   app_id          50872 non-null  int64  
 1   title           50872 non-null  object 
 2   date_release    50872 non-null  object 
 3   win             50872 non-null  bool   
 4   mac             50872 non-null  bool   
 5   linux           50872 non-null  bool   
 6   rating          50872 non-null  object 
 7   positive_ratio  50872 non-null  int64  
 8   user_reviews    50872 non-null  int64  
 9   price_final     50872 non-null  float64
 10  price_original  50872 non-null  float64
 11  discount        50872 non-null  float64
 12  steam_deck      50872 non-null  bool   
dtypes: bool(4), float64(3), int64(3), object(3)
memory usage: 3.7+ MB


#### app_id: 스팀 제품ID
#### title: 제목
#### date_release: 제품 출시일
#### win: 윈도우 지원 여부
#### mac: 맥 지원 여부
#### rating: 제품 추천 카테고리( Positive: 27%, Very positive: 26%, Other: 48%)
#### positive_ratio: positive 반응 비율
#### users_reviews: 유저 리뷰 개수(데이터셋에는 없음)
#### price_final: 할인 후에 미국 달러 가격
#### price_original: 할인 전에 가격
#### discount: 할인율
#### steam_deck: 스팀 덱 지원 여부 (스팀 게임기)

In [17]:
games_df.head()

,app_id,title,date_release,win,mac,linux,rating,positive_ratio,user_reviews,price_final,price_original,discount,steam_deck
0,13500,Prince of Persia: Warrior Within™,2008-11-21,True,False,False,Very Positive,84,2199,9.99,9.99,0.0,True
1,22364,BRINK: Agents of Change,2011-08-03,True,False,False,Positive,85,21,2.99,2.99,0.0,True
2,113020,Monaco: What's Yours Is Mine,2013-04-24,True,True,True,Very Positive,92,3722,14.99,14.99,0.0,True
3,226560,Escape Dead Island,2014-11-18,True,False,False,Mixed,61,873,14.99,14.99,0.0,True
4,249050,Dungeon of the ENDLESS™,2014-10-27,True,True,False,Very Positive,88,8784,11.99,11.99,0.0,True


In [18]:
games_df.dtypes.value_counts()

bool       4
int64      3
object     3
float64    3
dtype: int64

In [19]:
# object -> String으로 바꾸기
object_select = games_df.select_dtypes(include="object").columns
games_df[object_select] = games_df[object_select].astype('string')

In [20]:
# date_release -> datetime으로 바꾸기
games_df['date_release'] = pd.to_datetime(games_df['date_release'])

In [21]:
bool_column = ["win", "mac", "linux", "steam_deck"] # bool형 컬럼 1,0으로 교체
for column in bool_column:
    games_df[column] = games_df[column].replace({True:1, False:0})

In [22]:
# rating 컬럼 라벨 인코딩
rating_mapping = {
    'Overwhelmingly Positive': 8,
    'Very Positive': 7,
    'Mostly Positive': 6,
    'Positive': 5,
    'Mixed': 4,
    'Negative': 3,
    'Mostly Negative': 2,
    'Very Negative': 1,
    'Overwhelmingly Negative': 0
}

# replace 메소드를 사용하여 문자열을 숫자로 변환
games_df['rating'] = games_df['rating'].replace(rating_mapping)

ValueError: Cannot set non-string value '8' into a StringArray.

In [ ]:
games_df.dtypes.value_counts()

int64             8
float64           3
string            1
datetime64[ns]    1
dtype: int64

#### 결측치 확인

In [ ]:
games_df.isnull().sum()

app_id            0
title             0
date_release      0
win               0
mac               0
linux             0
rating            0
positive_ratio    0
user_reviews      0
price_final       0
price_original    0
discount          0
steam_deck        0
dtype: int64

#### 중복값 체크

In [ ]:
games_df.duplicated().sum()

0

In [ ]:
games_df.to_csv("../datasets/preprocessed_games.csv")

### Top 20 스팀 게임

In [ ]:
top_20_games = games_df[(games_df['positive_ratio'] >= 90) & (games_df['rating'] == 'Overwhelmingly Positive')].sort_values(by=['user_reviews', 'positive_ratio'], ascending=[False, True]).head(20)
top_20_games[['title','user_reviews','positive_ratio']]

,title,user_reviews,positive_ratio


In [ ]:
fig= px.scatter(top_20_games,x= 'title', y='price_final', title= 'Top 20 positive rated games on Steam', hover_name="title", hover_data={'title': False, 'date_release': True, 'user_reviews': True, 'positive_ratio': True}, labels={"title":"Title", "date_release":"Release Date", "price_final":"Price","positive_ratio": "Positive Ratio","user_reviews":"Total User Reviews"})
fig.update_yaxes(title_text='Price of game ($)')
fig.update_xaxes(title_text='Game Title')


fig.show()

### Worst 20 스팀 게임

In [ ]:
bottom_20_games = games_df[(games_df['positive_ratio'] <= 50) & (games_df['rating'].isin(['Overwhelmingly Negative', 'Negative','Mostly Negative']))].sort_values(by=['user_reviews', 'positive_ratio'], ascending=[False, True]).head(20)
bottom_20_games[['title','user_reviews','positive_ratio']]

,title,user_reviews,positive_ratio


In [ ]:
fig= px.scatter(bottom_20_games,x= 'title', y='price_final', title= 'Top 20 negative rated games on Steam', hover_name="title", hover_data={'title': False, 'date_release': True, 'user_reviews': True, 'positive_ratio': True}, labels={"title":"Game Title", "date_release":"Release Date", "price_final":"Price","positive_ratio": "Positive Ratio","user_reviews":"Total User Reviews"})
fig.update_yaxes(title_text='Price of game ($)')
fig.update_xaxes(title_text='Game Title')


fig.show()

### 게임 등급 분포

In [ ]:
rating_count = games_df['rating'].value_counts()
rating_count_df= pd.DataFrame(rating_count).reset_index()
rating_count_df.columns = ['Rating', 'Game Count']

fig= px.bar(rating_count_df, x='Game Count', y='Rating', color='Rating', text_auto= True, title= 'Game Rating Distribution', orientation='h')

fig.show()

### 리뷰 개수 분포

In [ ]:
def categorize_reviews(count):
    if count < 10000:
        return 'Less than 10,000'
    elif count < 100000:
        return '10,000 to 100,000'
    else:
        return 'More than 100,000'

games_df['review_category'] = games_df['user_reviews'].apply(categorize_reviews)
review_counts = games_df['review_category'].value_counts()
review_counts_df = pd.DataFrame(review_counts).reset_index()
review_counts_df.columns = ['Reviews', 'Game Count']

fig= px.bar(review_counts_df, x='Game Count', y='Reviews', text_auto= True, title= 'Game Review Distribution', orientation='h')

fig.show()

#### 추천 데이터

In [ ]:
rc_df = pd.read_csv("../datasets/recommendations.csv")
rc_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 38354101 entries, 0 to 38354100
Data columns (total 8 columns):
 #   Column          Dtype  
---  ------          -----  
 0   app_id          int64  
 1   helpful         int64  
 2   funny           int64  
 3   date            object 
 4   is_recommended  bool   
 5   hours           float64
 6   user_id         int64  
 7   review_id       int64  
dtypes: bool(1), float64(1), int64(5), object(1)
memory usage: 2.0+ GB


#### app_id: 제품 ID
#### helpful: 얼마나 많은 유저들이 helpful한 리뷰의 수
#### funny: 얼마나 많은 유저들이 funny한 리뷰의 수
#### date: 출시일
#### is_recommended: 사용자가 추천 했나 안했나
#### user_id: 유저 ID
#### review_id: 자동 생성 ID??

In [ ]:
rc_df.head(10)

,app_id,helpful,funny,date,is_recommended,hours,user_id,review_id
0,975370,0,0,2022-12-12,True,36.3,49625,0
1,304390,4,0,2017-02-17,False,11.5,2482,1
2,1085660,2,0,2019-11-17,True,336.5,243409,2
3,703080,0,0,2022-09-23,True,27.4,248701,3
4,526870,0,0,2021-01-10,True,7.9,22902,4
5,306130,0,0,2021-10-10,True,8.6,43700,5
6,238960,0,0,2017-11-25,True,538.8,84882,6
7,730,0,0,2021-11-30,False,157.5,60799,7
8,255710,0,0,2021-05-21,True,18.7,339767,8
9,289070,0,0,2020-05-26,True,397.5,435505,9


In [ ]:
rc_df.dtypes.value_counts()

int64      5
object     1
bool       1
float64    1
dtype: int64

In [ ]:
rc_df['date'] = rc_df["date"].astype("string")
rc_df["date"] = pd.to_datetime(rc_df["date"])

In [ ]:
# True를 1, False를 0으로 교체
rc_df["is_recommended"] = rc_df["is_recommended"].replace({True:1, False:0})

In [ ]:
rc_df.to_csv("../datasets/pre_recommendations.csv",index=False)

### 유저 데이터와 추천 데이터의 유저 아이디 겹치는 지 확인

In [ ]:
user_ids_in_user_df = set(users_df['user_id'])
user_ids_in_rc_df = set(rc_df['user_id'])

overlapping_user_ids = user_ids_in_user_df.intersection(user_ids_in_rc_df)

if len(overlapping_user_ids) > 0:
    print(f"There are {len(overlapping_user_ids)} overlapping user_ids.")
    print(f"Some overlapping user_ids: {list(overlapping_user_ids)[:10]}") 
else:
    print("There are no overlapping user_ids.")

There are 13253043 overlapping user_ids.
Some overlapping user_ids: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]


### 추천 데이터와 게임 데이터의 app_id가 겹치는 지 확인

In [ ]:
app_ids_in_games_df = set(games_df['app_id'])
app_ids_in_rc_df = set(rc_df['app_id'])

overlapping_app_ids = app_ids_in_games_df.intersection(app_ids_in_rc_df)

if len(overlapping_app_ids) > 0:
    print(f"There are {len(overlapping_app_ids)} overlapping app_ids")
    print(f"Some overlapping app_ids: {list(overlapping_app_ids)[:10]}")
else:
    print("There are no overlapping app_ids")

There are 33339 overlapping app_ids
Some overlapping app_ids: [393220, 1048580, 262150, 2097160, 10, 655370, 655380, 20, 393240, 2097180]


### 스팀 데이터

In [ ]:
steam=pd.read_csv("../datasets/steam.csv")
steam.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27075 entries, 0 to 27074
Data columns (total 18 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   appid             27075 non-null  int64  
 1   name              27075 non-null  object 
 2   release_date      27075 non-null  object 
 3   english           27075 non-null  int64  
 4   developer         27075 non-null  object 
 5   publisher         27075 non-null  object 
 6   platforms         27075 non-null  object 
 7   required_age      27075 non-null  int64  
 8   categories        27075 non-null  object 
 9   genres            27075 non-null  object 
 10  steamspy_tags     27075 non-null  object 
 11  achievements      27075 non-null  int64  
 12  positive_ratings  27075 non-null  int64  
 13  negative_ratings  27075 non-null  int64  
 14  average_playtime  27075 non-null  int64  
 15  median_playtime   27075 non-null  int64  
 16  owners            27075 non-null  object

#### appid: 아이디
#### name: 게임이름
#### release_date: 출시일
#### english: 영어지원여부
#### developer: 개발자(unique 값은 17113)
#### publisher: 출판자(unique 값은 14354)
#### platforms: 윈도우, 맥, 리눅스 등등 지원 여부(윈도우 68%, 윈,맥,리: 17% , 그외 나머지)
#### required_age: PEGI UK 기준에 따른 최소 필요 연령
#### categories: 세미콜론으로 구분된 게임 카테고리 single player, multi player,,
#### genres: 세미콜론으로 구분된 게임 장르
#### steamspy_tags: genres와 유사하지만 커뮤니티 투표로 장르?
#### achievements: 게임 내 성과의 수
#### positive_ratings: Number of positive ratings
#### negative_ratings: Number of negative ratings
#### average_playtime: 사용자 평균 플레이 시간
#### median_playtime: 사용자 중앙 플레이 시간 값
#### owners: 소유자 수 (0 ~ 20000: 69%, 20000 ~ 50000: 11%, Other: 20%)
#### price: 현재 가격

In [ ]:
# release_Date -> datetime 변환
# developer, publisher, name는 제거했다가 나중에 필요할 때 사용하는 식
# platforms -> 원핫하거나
# categories -> 원핫
# genres -> int
# steamspy_tags -> 원핫
# owners -> 원핫
steam.head()

,appid,name,release_date,english,developer,publisher,platforms,required_age,categories,genres,steamspy_tags,achievements,positive_ratings,negative_ratings,average_playtime,median_playtime,owners,price
0,10,Counter-Strike,2000-11-01,1,Valve,Valve,windows;mac;linux,0,Multi-player;Online Multi-Player;Local Multi-P...,Action,Action;FPS;Multiplayer,0,124534,3339,17612,317,10000000-20000000,7.19
1,20,Team Fortress Classic,1999-04-01,1,Valve,Valve,windows;mac;linux,0,Multi-player;Online Multi-Player;Local Multi-P...,Action,Action;FPS;Multiplayer,0,3318,633,277,62,5000000-10000000,3.99
2,30,Day of Defeat,2003-05-01,1,Valve,Valve,windows;mac;linux,0,Multi-player;Valve Anti-Cheat enabled,Action,FPS;World War II;Multiplayer,0,3416,398,187,34,5000000-10000000,3.99
3,40,Deathmatch Classic,2001-06-01,1,Valve,Valve,windows;mac;linux,0,Multi-player;Online Multi-Player;Local Multi-P...,Action,Action;FPS;Multiplayer,0,1273,267,258,184,5000000-10000000,3.99
4,50,Half-Life: Opposing Force,1999-11-01,1,Gearbox Software,Valve,windows;mac;linux,0,Single-player;Multi-player;Valve Anti-Cheat en...,Action,FPS;Action;Sci-fi,0,5250,288,624,415,5000000-10000000,3.99


In [ ]:
steam["platforms"].value_counts()

windows              18398
windows;mac;linux     4623
windows;mac           3439
windows;linux          610
mac                      3
mac;linux                1
linux                    1
Name: platforms, dtype: int64

In [ ]:
steam["categories"].value_counts()

Single-player                                                                                                              6110
Single-player;Steam Achievements                                                                                           2334
Single-player;Steam Achievements;Steam Trading Cards                                                                        848
Single-player;Partial Controller Support                                                                                    804
Single-player;Steam Trading Cards                                                                                           792
                                                                                                                           ... 
Single-player;Steam Achievements;Steam Trading Cards;Captions available;Partial Controller Support;Commentary available       1
Single-player;Steam Achievements;Full controller support;Steam Trading Cards;VR Support;Steam Cloud;Stea

In [ ]:
steam["steamspy_tags"].value_counts()

Action;Indie;Casual                      845
Action;Adventure;Indie                   714
Early Access;Action;Indie                507
Adventure;Indie;Casual                   442
Indie;Casual                             378
                                        ... 
Action;Adventure;Warhammer 40K             1
Anime;RPG;Cute                             1
Action;Casual;Tower Defense                1
RPG;Turn-Based;Co-op                       1
Early Access;Adventure;Sexual Content      1
Name: steamspy_tags, Length: 6423, dtype: int64

In [ ]:
def one_hot_encode_multilabel(dataframe, column):
    # ;으로 나누기
    split_items = dataframe[column].str.split(';')
    
    # 각 고윳값 라벨
    unique_labels = set(item for sublist in split_items for item in sublist)

    # 각 라벨에 대해 열 생성
    for label in unique_labels:
        dataframe[label] = split_items.apply(lambda x: int(label in x))

    return dataframe.drop(column, axis=1)

def categorize_range(range_str):
    # 범위를 -를 기준으로 설정
    lower, upper = map(int, range_str.split('-'))
    if lower >= 0 and upper <= 20000:
        return 0
    elif lower >= 20000 and upper <= 50000:
        return 1
    else:
        return 2

In [ ]:
columns = ["platforms", "categories", "genres","steamspy_tags"]
# 각 컬럼에 대해 원핫 인코딩
for column in columns:
    steam = one_hot_encode_multilabel(steam, column)

C:\Users\gihun\AppData\Local\Temp\ipykernel_20272\1953474591.py:10: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`



In [ ]:
steam['owners'] = steam['owners'].apply(categorize_range)

In [ ]:
steam["release_date"] = pd.to_datetime(steam["release_date"])

In [ ]:
steam.to_csv("../datasets/pre_steam.csv", index=False)

### 스팀 태그 데이터

In [ ]:
steam_tag=pd.read_csv("../datasets/steamspy_tag_data.csv")
steam_tag.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29022 entries, 0 to 29021
Columns: 372 entries, appid to e_sports
dtypes: int64(372)
memory usage: 82.4 MB


In [ ]:
steam_tag.head()

,appid,1980s,1990s,2.5d,2d,2d_fighter,360_video,3d,3d_platformer,3d_vision,...,warhammer_40k,web_publishing,werewolves,western,word_game,world_war_i,world_war_ii,wrestling,zombies,e_sports
0,10,144,564,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,550
1,20,0,71,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,30,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,5,122,0,0,0
3,40,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,50,0,77,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
# 모두 int형
steam_tag.dtypes.value_counts()

int64    372
dtype: int64

#### 결측값 확인

In [ ]:
steam_tag.isnull().sum()

appid           0
1980s           0
1990s           0
2.5d            0
2d              0
               ..
world_war_i     0
world_war_ii    0
wrestling       0
zombies         0
e_sports        0
Length: 372, dtype: int64

#### 중복값 확인

In [ ]:
steam_tag.duplicated().sum()

0

#### 스팀 데이터, 게임 데이터 겹치는 지 확인

In [ ]:
app_ids_in_games_df = set(games_df['app_id'])
app_ids_in_steam = set(steam['appid'])

overlapping_app_ids = app_ids_in_games_df.intersection(app_ids_in_steam)

if len(overlapping_app_ids) > 0:
    print(f"There are {len(overlapping_app_ids)} overlapping app_ids")
    print(f"Some overlapping app_ids: {list(overlapping_app_ids)[:10]}")
else:
    print("There are no overlapping app_ids")

There are 18749 overlapping app_ids
Some overlapping app_ids: [491520, 32770, 688130, 393220, 65540, 262150, 360450, 458760, 753670, 10]
